In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Documents/Personal/Education/FRI/Masters/1.Letnik/1.Semester/SB/IBB-project-2/

/content/drive/MyDrive/Documents/Personal/Education/FRI/Masters/1.Letnik/1.Semester/SB/IBB-project-2


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import glob
import cv2
import keras

In [ ]:
TRAIN_IMAGES_PATH = "dataset/train/images/"
TRAIN_ANNOTATIONS = "dataset/train/annotations/"

### 1. Image preporcessing and preparation



In [ ]:
train_images=[]
train_labels=[]

In [ ]:
def calculate_iou(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [ ]:
selectiveSearch = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [ ]:
# Retrieve all images paths (sorted)
list_of_files = list(filter(os.path.isfile, glob.glob(TRAIN_IMAGES_PATH + '*')))
list_of_files.sort(key=lambda f: int(re.sub('\D', '', f)))

# Iterate over all images in train dataset and calculate IoU
for file in list_of_files[:200]:

    # Read annotation file of image using pandas
    annotation_file = TRAIN_ANNOTATIONS + file.split("/")[-1].split(".")[0] + ".csv"
    annotation_df = pd.read_csv(annotation_file, sep=" ", skiprows=[0], names=["x1", "y1", "x2", "y2"])

    # Read current image
    image = cv2.imread(file)

    # Apply image to selective search algorithm and get results
    selectiveSearch.setBaseImage(image)                 # Setting base image
    selectiveSearch.switchToSelectiveSearchFast()       # Running selective search
    results = selectiveSearch.process()                 # Processing selective search and retrieve results
    
    image_copy = image.copy()

    # Counting number of positive and negative region (add 30 each)
    num_positive = 0
    num_negative = 0

    # Iterate over first 2000 results
    for idx, result in enumerate(results[:2000]):
        x, y, w, h = result

        # Proposed area annotation
        result_annotation = {
            "x1": x,
            "x2": x + w,
            "y1": y,
            "y2": y + h
        }

        # For each annotation of current image we calulate the IoU
        for _, annotation in annotation_df.iterrows():

            # Calculate IoU between ground truth annotation region and proposed region
            iou = calculate_iou(annotation, result_annotation)

            # If IoU is larger than 0.7 we add the add the region as positive
            if iou > 0.7 and num_positive < 30:
                trimmed_image = image_copy[x:x+w, y:y+h]
                resized_image = cv2.resize(trimmed_image, (224, 224), interpolation = cv2.INTER_AREA)
                train_images.append(resized_image)

                train_labels.append(1)      # mark as 1
                num_positive += 1
            
            # If IoU is smaller than 0.3 we add the add the region as negative
            if iou < 0.7 and num_negative < 30:
                trimmed_image = image_copy[x:x+w, y:y+h]
                resized_image = cv2.resize(trimmed_image, (224, 224), interpolation = cv2.INTER_AREA)
                train_images.append(resized_image)

                train_labels.append(0)      # mark as 0
                num_negative += 1

        if num_positive >= 30 and num_negative >= 30:
            break

In [ ]:
X_new = np.array(train_images)
y_new = np.array(train_labels)

In [ ]:
np.save("X_new", X_new)
np.save("y_new", y_new)